In [ ]:
import asyncio
from datetime import datetime
import os

from astropy.time import Time
from bokeh.models import DatetimeTickFormatter
import holoviews as hv
from holoviews import opts
hv.extension('bokeh', logo=False)
import numpy as np
import pandas as pd
import tables as tb

from lsst_efd_client import EfdClient

In [ ]:
ipath = os.path.expanduser("~/Dropbox_LSST/Dome_Seeing_Monitor/DSM_Data/20200128")
date_str = "20200129_010709"

In [ ]:
ifname_psd = os.path.join(ipath, f"smm_psd_{date_str}.h5")

# Get data keys from file
psd_h5 = tb.open_file(ifname_psd)
glist = psd_h5.root._f_list_nodes(classname="Group")
keys = [k._v_name for k in glist]

In [ ]:
datetimes = []
for key in keys:
    datetimes.append(datetime.strptime(key.replace('DT_', ''), '%Y%m%d_%H%M%S'))
datetimes = datetimes[2:-3]

In [ ]:
client = EfdClient('summit_efd')

In [ ]:
start = Time(datetimes[0])
end = Time(datetimes[-1])
df = await client.select_time_series('lsst.sal.DSM.domeSeeing', '*', start.tai, end.tai, 1)
df_w = await client.select_time_series('lsst.sal.Environment.windSpeed', '*', start.tai, end.tai, 1)

In [ ]:
src = hv.Table(df.reset_index())
src_w = hv.Table(df_w.reset_index())
dt_format = '%F %T'
formats = {'days': dt_format, 'months': dt_format, 'hours': dt_format, 'minutes': dt_format}
tick_rotation = 75

In [ ]:
x_tuple = ('index', 'Date')
full_width = 350
end_col_size = full_width - 75
date_formatter = DatetimeTickFormatter(**formats)

cx = hv.Curve(src, x_tuple, ('centroidX', 'X'), label='X')
cy = hv.Curve(src, x_tuple, ('centroidY', 'Y'), label='Y')
centroids = (cx * cy).opts(ylabel='Pixel Position', title='Centroids', width=full_width)
sx = hv.Curve(src, x_tuple, ('rmsX', 'X'), label='X')
sy = hv.Curve(src, x_tuple, ('rmsY', 'Y'), label='Y')
seeing = (sx * sy).opts(ylabel='scaled RMS (arcsec)', title='Seeing', width=full_width)
wind = hv.Curve(src_w, x_tuple, ('avg2M', 'Speed (m/s)'), label='Wind').opts(width=end_col_size)
flux = hv.Curve(src, x_tuple, ('flux', 'Flux'), label='Flux').opts(ylabel='')
max_adc = hv.Curve(src, x_tuple, ('maxADC', 'Max ADC'), label='Max ADC').opts(ylabel='')
fwhm = hv.Curve(src, x_tuple, ('fwhm', '(pixels)'), label='FWHM').opts(width=end_col_size)

In [ ]:
layout = (centroids + seeing + wind + flux + max_adc + fwhm).cols(3)
layout.opts(opts.Layout(toolbar='above'),
            opts.Overlay(legend_position='right'),
            opts.Curve(xformatter=date_formatter, xrotation=tick_rotation, height=400))